# Assignment_1
## Feature sign search algorithm

##### A:2D Matrix of basis vectors
##### y:input vector to be sparsely encoded
##### gamma: sparsity co-efficient
##### x: output sparse encoding of y on the basis A with sparsity co-efficient gamma


In [10]:
import numpy as np

def feature_sign_search(A, y, gamma):
    
    effective_zero = 1e-18
    
    gram_matrix = np.dot(A.T, A)

    target_correlation = np.dot(A.T, y)
   
    x = np.zeros(gram_matrix.shape[0])
    theta = np.zeros(gram_matrix.shape[0], dtype=np.int8)
    active_set = set()
    z_opt = np.inf
    nz_opt = 0
    grad = - 2 * target_correlation  # + 2 * np.dot(gram_matrix, x)
    max_grad_zero = np.argmax(np.abs(grad))
    
    sds = np.dot(y.T, y)
    while z_opt > gamma or not np.allclose(nz_opt, 0):
        if np.allclose(nz_opt, 0):
            candidate = np.argmax(np.abs(grad) * (theta == 0))
            if grad[candidate] > gamma:
                theta[candidate] = -1.
                x[candidate] = 0.
                active_set.add(candidate)
            elif grad[candidate] < -gamma:
                theta[candidate] = 1.
                x[candidate] = 0.
                active_set.add(candidate)
            if len(active_set) == 0:
                break
        indices = np.array(sorted(active_set))
        restr_gram = gram_matrix[np.ix_(indices, indices)]
        restr_corr = target_correlation[indices]
        restr_sign = theta[indices]
        rhs = restr_corr - gamma * restr_sign / 2
        new_x = np.linalg.solve(np.atleast_2d(restr_gram), rhs)
        new_theta = np.sign(new_x)
        restr_oldsol = x[indices]
        sign_flips = np.where(abs(new_theta - restr_sign) > 1)[0]
        if len(sign_flips) > 0:
            best_obj = np.inf
            best_curr = None
            best_curr = new_x
            best_obj = (sds + (np.dot(new_x,
                                      np.dot(restr_gram, new_x))
                        - 2 * np.dot(new_x, restr_corr))
                        + gamma * abs(new_x).sum())
            for idx in sign_flips:
                a = new_x[idx]
                b = restr_oldsol[idx]
                prop = b / (b - a)
                curr = restr_oldsol - prop * (restr_oldsol - new_x)
                cost = sds + (np.dot(curr, np.dot(restr_gram, curr))
                              - 2 * np.dot(curr, restr_corr)
                              + gamma * abs(curr).sum())
                if cost < best_obj:
                    best_obj = cost
                    best_prop = prop
                    best_curr = curr
        else:
            best_curr = new_x
        x[indices] = best_curr
        zeros = indices[np.abs(x[indices]) < effective_zero]
        x[zeros] = 0.
        theta[indices] = np.int8(np.sign(x[indices]))
        
        active_set.difference_update(zeros)
        grad = - 2 * target_correlation + 2 * np.dot(gram_matrix, x)
        if len(grad[theta == 0]) == 0:
             break
        z_opt = np.max(abs(grad[theta == 0]))
        nz_opt = np.max(abs(grad[theta != 0] + gamma * theta[theta != 0]))
    return x

#### Example

In [11]:
input = np.array([1,2,3,4,5])
basis = np.array([[0,0,0,0,5],[1,0,0,0,0],[0,1,0,0,0],[0,1.5,0,0,0],[0,0,3,4,0],[0,0,0,1,0],[0,0,0,0,1]]).T
sparsity = 0.01
sparse_encoding = feature_sign_search(basis,input,sparsity)
print(u'Original input vector\n')
print(input)
print(u'Reconstructed vector\n')
print(np.matmul(basis,sparse_encoding))
print(u'Sparse representation\n')
print(sparse_encoding)

Original input vector

[1 2 3 4 5]
Reconstructed vector

[ 0.995       1.99666667  2.9994      3.9992      4.999     ]
Sparse representation

[ 0.9998      0.995       0.          1.33111111  0.9998      0.          0.        ]
